In [1]:
items = {1,2,3,4,5}
c = {1:50, 2:40, 3:10, 4:70, 5:55}
w = {1:7, 2:5, 3:1, 4:9, 5:6}
capacity = 15

ratio = {j:c[j]/w[j] for j in items}
sItems = [key for key, val in sorted(ratio.items(),
                                      key=lambda s:s[1], reverse=True)]
for j in sItems:
  print('c[%d]/w[%d] = %lf' % (j,j,c[j]/w[j]))

c[3]/w[3] = 10.000000
c[5]/w[5] = 9.166667
c[2]/w[2] = 8.000000
c[4]/w[4] = 7.777778
c[1]/w[1] = 7.142857


In [3]:
x={j:0 for j in sItems}
cap=capacity
for j in sItems:
  if w[j] <= cap:
    cap -= w[j]
    x[j] = 1

print(x)
print('総価格 = ', sum(c[j]*x[j] for j in sItems))

{3: 1, 5: 1, 2: 1, 4: 0, 1: 0}
総価格 =  105


In [4]:
x={j:0 for j in sItems}
cap = capacity
for j in sItems:
  if w[j] <= cap:
    cap -= w[j]
    x[j] = 1
  else:
    x[j] = cap/w[j]
    break
print(x)
print('総価格 = ', sum(c[j]*x[j] for j in sItems))

{3: 1, 5: 1, 2: 1, 4: 0.3333333333333333, 1: 0}
総価格 =  128.33333333333334


In [6]:
class KnapsackProblem(object):
  """ The difinition of KnapsackProblem """
  def __init__(self, name, capacity, items, costs, weights,
               zeros=set(), ones=set()):
      self.name = name
      self.capacity = capacity
      self.items = items
      self.costs = costs
      self.weights = weights
      self.zeros = zeros
      self.ones = ones
      self.lb = -100
      self.ub = -100
      ratio = {j:costs[j]/weights[j] for j in items}
      self.sitemList = [k for k, v in
           sorted(ratio.items(), key=lambda x:x[1], reverse=True)]
      self.xlb = {j:0 for j in self.items}
      self.xub = {j:0 for j in self.items}
      self.bi = None
  
  def getbounds(self):
    """ Calculate the upper and lower bounds, """
    for j in self.zeros:
      self.xlb[j] = self.xub[j] = 0
    for j in self.ones:
      self.xlb[j] = self.xub[j] = 1
    if self.capacity < sum(self.weights[j] for j in self.ones):
      self.lb = self.ub = -100
      return 0

    ritems = self.items - self.zeros - self.ones
    sitems = [j for j in self.sitemList if j in ritems]
    cap = self.capacity - sum(self.weights[j] for j in self.ones)
    for j in sitems:
      if self.weights[j] <= cap:
        cap -= self.weights[j]
        self.xlb[j] = self.xub[j] = 1
      else:
        self.xub[j] = cap/self.weights[j]
        self.bi = j
        break
    self.lb = sum(self.costs[j]*self.xlb[j] for j in self.items)
    self.ub = sum(self.costs[j]*self.xub[j] for j in self.items)
  
  def __str__(self):
    """ KnapsackProblem の情報を印字 """
    return('Name = '+self.name+', capacity = '+str(self.capacity)+',\n'
         'items = '+str(self.items)+',\n'+
         'costs = '+str(self.costs)+',\n'+
         'weights = '+str(self.weights)+',\n'+
         'zeros = '+str(self.zeros)+', ones = '+str(self.ones)+',\n'+
         'lb = '+str(self.lb)+', ub = '+str(self.ub)+',\n'+
         'sitemList = '+str(self.sitemList)+',\n'+
         'xlb = '+str(self.sitemList)+',\n'+
         'bi = '+str(self.bi)+',\n')

In [7]:
from pulp import *

def KnapsackProblemSolve(capacity, items, costs, weights):
  from collections import deque
  queue = deque()
  root = KnapsackProblem('KP', capacity=capacity,
       items = items, costs = costs, weights = weights,
       zeros = set(), ones = set())
  root.getbounds()
  best = root
  queue.append(root)
  while queue != deque([]):
    p = queue.popleft()
    p.getbounds()
    if p.ub > best.lb:
      if p.lb > best.lb:
        best = p
      if p.ub > p.lb:
        k = p.bi
        p1 = KnapsackProblem(p.name+'+'+str(k),
           capacity = p.capacity, items = p.items,
           costs = p.costs, weights = p.weights,
           zeros = p.zeros, ones = p.ones.union({k}))
        queue.append(p1)
        p2 = KnapsackProblem(p.name+'-'+str(k),
             capacity = p.capacity, items = p.items,
             costs = p.costs, weights = p.weights,
             zeros = p.zeros.union({k}), ones = p.ones)
        queue.append(p2)
  return 'Optimal', best.lb, best.xlb

In [8]:
capacity = 15
items = {1,2,3,4,5}
c = {1:50, 2:40, 3:10, 4:70, 5:55}
w = {1:7, 2:5, 3:1, 4:9, 5:6}

res = KnapsackProblemSolve(capacity=capacity,
                      items=items, costs=c, weights=w)
print('Optimal value = ', res[1])
print('Optimal solution = ', res[2])

Optimal value =  125
Optimal solution =  {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}


In [13]:
from pulp import *
import numpy as np
MEPS = 1.0e-8

def binpacking(capacity, w):
  m = len(w)
  items = set(range(m))
  A = np.identity(m)

  solved = False
  columns = 0
  dual = LpProblem(name='D(K)', sense=LpMaximize)
  y = [LpVariable('y'+str(i), lowBound=0) for i in items]

  dual += lpSum(y[i] for i in items)
  for j in range(len(A.T)):
    dual += lpDot(A.T[j],y) <= 1, 'ineq'+str(j)

  while not(solved):
    dual.solve()

    costs = {i: y[i].varValue for i in items}
    weights = {i: w[i] for i in items}
    (state, val, sol) = KnapsackProblemSolve(capacity, items, costs, weights)

    if val >= 1.0+MEPS:
      a = np.array([int(sol[i]) for i in items])
      dual += lpDot(a, y) <= 1, 'ineq'+str(m+columns)
      A = np.hstack((A, a.reshape((-1,1))))
      columns += 1
    else:
      solved = True

  print('Generated columns: ', columns)
  m,n = A.shape
  primal = LpProblem(name='P(K)', sense=LpMinimize)
  x = [LpVariable('x'+str(j), lowBound=0, cat='Binary') for j in range(n)]

  primal += lpSum(x[j] for j in range(n))
  for i in range(m):
    primal += lpDot(A[i], x) >= 1, 'ineq'+str(i)

  primal.solve()
  if value(primal.objective) - value(dual.objective) < 1.0-MEPS:
    print('Optimal solution found: ')
  else:
    print('Approximated solution found: ')
  K = [j for j in range(n) if x[j].varValue > MEPS]
  result = []
  itms = set(range(m))
  for j in K:
    J = {i for i in range(m) if A[i,j] > MEPS and i in itms}
    r = [w[i] for i in J]
    itms -= J
    result.append(r)
  print(result)

In [14]:
capacity = 25
items = set(range(10))
np.random.seed(1)
w = {i:np.random.randint(5,10) for i in items}
w2 = [w[i] for i in items]
print(w2)

binpacking(capacity, w)

[8, 9, 5, 6, 8, 5, 5, 6, 9, 9]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/156b4c47a6014cd198f64e23ccc3f737-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/156b4c47a6014cd198f64e23ccc3f737-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 36 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 10 rows, 10 columns and 10 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-10) rows, 0 (-10) columns and 0 (-10) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 10
After Postsolve, objective 10, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 10 - 0 iterations time 0.002, Presolv

In [19]:
from pulp import *
MEPS = 1.0e-8

def binpacking2(capacity, w):
  n = len(w)
  items = range(n)
  bpprob = LpProblem(name='BinPacking2', sense=LpMinimize)
  z = [LpVariable('z'+str(j), lowBound=0, cat='Binary') for j in items]
  x = [[LpVariable('x'+str(i)+str(j), lowBound=0, cat='Binary') for j in items] for i in items]

  bpprob += lpSum(z[i] for i in items)
  for i in items:
    bpprob += lpSum(x[i][j] for j in items) == 1
  for j in items:
    bpprob += lpSum(x[i][j]*w[i] for i in items) <= capacity*z[j]

  bpprob.solve()
  result = []
  for j in items:
    if z[j].varValue > MEPS:
      r = [w[i] for i in items if x[i][j].varValue > MEPS]
      result.append(r)
  print(result)

In [20]:
capacity = 25
items = set(range(10))
np.random.seed(1)
w = {i:np.random.randint(5,10) for i in items}
w2 = [w[i] for i in items]
print(w2)

binpacking2(capacity,w)

[8, 9, 5, 6, 8, 5, 5, 6, 9, 9]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/bc95784b37cf4013b5acc54a5c24e228-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/bc95784b37cf4013b5acc54a5c24e228-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25 COLUMNS
At line 466 RHS
At line 487 BOUNDS
At line 598 ENDATA
Problem MODEL has 20 rows, 110 columns and 210 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.8 - 0.00 seconds
Cgl0004I processed model has 20 rows, 110 columns (110 integer (110 of which binary)) and 210 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 7 integers unsatisfied sum - 2.18667
Cbc0